### Musterlösung zu den Kontrollfragen des vorherigen Notebooks

* Ist der folgende Satz korrekt? "Der Schutz der Privacy ist besser, je kleiner das Privacy-Budget und je grösser das Delta."
> Je mehr sich das Delta an 0 annähert, desto näher kommt die Funktion an die reine und striktere Differential Privacy. Der Satz ist also falsch. Richtig wäre: "Der Schutz der Privacy ist besser, je kleiner das Privacy-Budget und je **kleiner** das Delta."

* Wie wird die Gauss Verteilung auch genannt?
> Normalverteilung

* Eine Person kommt potentiell in sehr vielen Statistiken (basierend auf demselben Mechanismus) vor. Wähle ich besser den Laplace oder den Gauss Mechanismus?
> Den Gauss Mechanismus, da bei diesem das Rauschen geringer sein wird.

<hr>

# Notebook 5: Der exponentielle Mechanismus

***Hier geht es zum vorherigen Notebook dieser Serie: [Notebook 4: Der Gauss Mechanismus](./4_Gauss-Mechanismus.ipynb)***

## Lernziele

Folgende Lernziele sollten mit der Bearbeitung dieses Jupyter Notebooks erreicht werden können:
- Die Teilnehmenden sind in der Lage, die Eigenschaften der Differential Privacy für eine einfache Funktion mittels des exponentiellen Mechanismus zu implementieren.

## Definition des exponentiellen Mechanismus

Der exponentielle Mechanismus ist besonders geeignet für Anwendungen, bei welchen kein Rauschen zur Ausgabe selbst hinzugefügt werden kann, wie z.B. bei **nicht-numerischen Abfragen**. Beispielsweise wenn die Krankheit ausgegeben werden soll, welche am häufigsten in der Datensammlung vorkommt. Ist die Antwort "Diabetes", kann ja zu diesem String nicht einfach eine Zufallszahl dazuaddiert werden. Aus diesem Grund kommt eine Bewertungsfunktion zum Einsatz, welche die möglichen Antworten bewertet. Basierend auf diesen Bewertungen wird anschliessend für jede mögliche Ausgabe eine Wahrscheinlichkeit berechnet, mit welcher diese Ausgabe vom Mechanismus ausgegeben werden soll.

Die Art der Bewertungsfunktion ist abhängig vom Anwendungsfall und muss entsprechend spezifisch auf den Anwendungsfall passend implementiert werden.

<div class="alert alert-success">
<b>Definition: Exponentieller Mechanismus</b>
    <br />
    <br /> 
Es sei die Bewertungsfunktion $q(D, \phi)$ gegeben. Der Mechanismus $M(D)$ erfüllt die Eigenschaften von $\varepsilon$-Differential Privacy, wenn dieser die Ausgabe $\phi \in \Phi$ mit der Wahrscheinlichkeit propotional zu

$$exp({\frac{\varepsilon * q(D, \phi)}{2 * s}})$$

ausgibt. Wobei $s$ die Sensitivität der Bewertungsfunktion und $exp()$ die Eulersche Exponentialfunktion ist.
</div>

Die Definition muss nicht im Detail verstanden werden. Wichtig ist es, dass das Prinzip des exponentiellen Mechanismus verstanden wird. Zum besseren Verständnis wird der Ablauf in <a href="#abb-1">Abbildung 1</a> grafisch gezeigt.

<br>
<br>
<center>
<img src="./src/Exponentieller-Mechanismus.png" width="50%" alt="Funktionsweise Exponentieller Mechanismus">
<br>
<br>
<a name="abb-1">Abbildung 1: Funktionsweise des exponentiellen Mechanismus</center></a>

Zur Veranschaulichung des exponentiellen Machanismus folgend ein Beispiel. In <a href="#tab-1">Tabelle 1</a> sind Krankheiten und die Anzahl erkrankter Personen aufgeführt. Es soll nun ein Mechanismus implementiert werden, der die häufigste Krankheit in der Datensammlung ausgibt. Die Bewertungsfunktion wird so definiert, dass diese der Anzahl Personen entspricht. "Diabetes" wird also eine Bewertung von "24" zugeteilt, "Hepatitis" eine Bewertung von "8" usw. Die Sensitivität der Bewertungsfunktion ist 1, da es sich um eine Zählfunktion handelt.

Nun werden die Wahrscheinlichkeiten für die Epsilon-Werte 0.0, 0.1 und 1.0 berechnet:

| Krankheit | Anzahl Personen | $\varepsilon$ = 0.0 | $\varepsilon$ = 0.1 | $\varepsilon$ = 1.0 |
| :- | :-: | :-: | :-: | :-: |
| Diabetes | 24 | 0.25 | 0.32 | 0.12 |
| Hepatitis | 8 | 0.25 | 0.15 | 4 * 10$^{-5}$|
| Grippe | 28 | 0.25 | 0.40 | 0.88|
| HIV | 5 | 0.25 | 0.13 | 8.9 * 10$^{-6}$|

<br>
<a name="tab-1"><center>Tabelle 1: Wahrscheinlichkeiten bei verschiedenen $\varepsilon$-Werten</center></a>

Bei einem Epsilon von 0 wird jede Krankheit mit derselben Wahrscheinlichkeit (25%) ausgegeben. Dies hat zwar den höchsten Schutz zur Folge, reduziert aber sehr stark den Nutzen der Daten. Bei einem Epsilon-Wert von 0.1 wird zu 40% als Antwort "Grippe" ausgegeben, aber auch zu 32% kann die Antwort "Diabetes" sein. So sind Nutzen der Daten und der Schutz der Privacy ziemlich gut ausbalanciert. Bei einem Epsilon-Wert von 1 unterscheiden sich die Wahrscheinlichkeiten der Krankheiten signifikant. Zu 88% wird das Resultat "Grippe" sein. Dadurch wird zwar ein grosser Nutzen der Daten erreicht, jedoch auf Kosten der Privacy.

## Implementation des exponentiellen Mechanismus

Das oben eingeführte Beispiel mit den Krankheiten und der Anzahl erkrankter Personen wird nun implementiert. Das Ziel ist es also einen Mechanismus zu implementieren, welcher unter Einhaltung der Differential Privacy die häufigste Krankheit ausgibt. 

Nachfolgend werden die sensiblen (und frei erfundenen!) Daten in einem Array gespeichert:

In [1]:
from array import *

datensammlung = [["Jean","Müller","Hepatitis"],
                ["Sarah","Rochat","Diabetes"],
                ["Elisabeth","Baumann","Diabetes"],
                ["Marianne","Frei","Diabetes"],
                ["Rosa","Da Silva","Grippe"],
                ["Juan","Meier","Grippe"],
                ["Anita","Weber","HIV"],
                ["Anna","Schneider","Grippe"],
                ["Rita","Rochat","Diabetes"],
                ["Monika","Fischer","Grippe"],
                ["Loris","Schneider","Diabetes"],
                ["Manuela","Meyer","Grippe"],
                ["Eva","Zürcher","Grippe"],
                ["Esther","Rochat","HIV"],
                ["Beatrice","Meyer","Grippe"],
                ["Dominique","Fischer","Hepatitis"],
                ["Verena","Elmer","Diabetes"],
                ["Michele","Wyss","Grippe"],
                ["Sonja","Krasniqi","Grippe"],
                ["Daniele","Wyss","Diabetes"],
                ["Nicole","Bucher","Diabetes"],
                ["Sara","Bernasconi","Grippe"],
                ["Sandra","Elmer","Grippe"],
                ["Robin","Zürcher","Diabetes"],
                ["Ruth","Nguyen","Diabetes"],
                ["Jacqueline","Caduff","Grippe"],
                ["Susanne","Caduff","Grippe"],
                ["Claudia","Martin","Diabetes"],
                ["Maria","Favre","Diabetes"],
                ["Barbara","Huber","Hepatitis"],
                ["Ana","Schmid","Grippe"],
                ["Laura","Weber","HIV"],
                ["Daniela","Moser","Diabetes"],
                ["Maxime","Moser","HIV"],
                ["Alex","Müller","Grippe"],
                ["Heidi","Da Silva","Hepatitis"],
                ["Claude","Frei","Grippe"],
                ["Karin","Keller","Grippe"],
                ["Nicola","Brunner","Grippe"],
                ["Noah","Bernasconi","Grippe"],
                ["Marie","Martin","Diabetes"],
                ["Elia","Bucher","Grippe"],
                ["Doris","Brunner","HIV"],
                ["Ursula","Nguyen","Grippe"],
                ["Ali","Meier","Diabetes"],
                ["Katharina","Gerber","Grippe"],
                ["Irene","Krasniqi","Grippe"],
                ["Jan","Nguyen","Diabetes"],
                ["Margrit","Steiner","Grippe"],
                ["Andrea","Keller","Hepatitis"],
                ["Willy","Müller","Hepatitis"],
                ["Erika","Bernasconi","Diabetes"],
                ["Rui","Caduff","Grippe"],
                ["Luca","Huber","Diabetes"],
                ["Nino","Baumann","Hepatitis"],
                ["Janis","Elmer","Diabetes"],
                ["Brigitte","Steiner","Diabetes"],
                ["Yannick","Meier","Grippe"],
                ["Silvia","Da Silva","Diabetes"],
                ["Andrea","Gerber","Diabetes"],
                ["Sascha","Zürcher","Hepatitis"],
                ["Rosmarie","Krasniqi","Diabetes"],
                ["Julia","Schmid","Grippe"],
                ["Stéphane","Krasniqi","Grippe"],
                ["Yvonne","Schmid","Diabetes"]]

Nachfolgend wird die Funktion `countDiseases` implementiert, welche die Krankheiten in der Datensammlung zählt.

In [2]:
def countDiseases(array):
    
    diabetes = 0
    hepatitis = 0
    grippe = 0
    hiv = 0
    
    counts = [diabetes, hepatitis, grippe, hiv]
    
    for x in array:
        if x[2] == "Diabetes":
            counts[0] += 1
        elif x[2] == "Hepatitis":
            counts[1] += 1
        elif x[2] == "Grippe":
            counts[2] += 1
        elif x[2] == "HIV":
            counts[3] += 1
        else:
            pass
    return counts

Zur Kontrolle geben wir die Anzahl der erkrankten Personen einmal aus.

In [3]:
print("Diabetes: " + str(countDiseases(datensammlung)[0]))
print("Hepatitis: " + str(countDiseases(datensammlung)[1]))
print("Grippe: " + str(countDiseases(datensammlung)[2]))
print("HIV: " + str(countDiseases(datensammlung)[3]))

Diabetes: 24
Hepatitis: 8
Grippe: 28
HIV: 5


Wir sehen, dass die Zahlen identisch sind mit jenen in <a href="#tab-1">Tabelle 1</a>. Wie im Beispiel erklärt, entspricht in diesem Fall die Funktion `countDiseases` auch gleich der Bewertungsfunktion. Entsprechend muss an dieser Stelle keine eigene Implementierung der Bewertungsfunktion mehr gemacht werden, sondern es kann dafür die Funktion `countDiseases` verwendet werden. Wir führen diese Funktion nun einmal aus und speichern die einzelnen Bewertungen im Array `scores` ab.

In [4]:
scores = countDiseases(datensammlung)

Nun werden die Wahrscheinlichkeiten für jedes Element berechnet und im Array `probabilities` gespeichert. Es wird ein $\varepsilon$ von 0.1 und die Sensitivität von 1 verwendet.

In [5]:
import numpy as np

epsilon = 0.1
sensitivity = 1

probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]

Zur Verifizierung wird der Inhalt des Arrays `probabilities` ausgegeben:

In [6]:
print(probabilities)

[3.320116922736548, 1.4918246976412703, 4.055199966844675, 1.2840254166877414]


Es ist ersichtlich, dass die Werte zum einen nicht mit jenen aus der Tabelle übereinstimmen und zum anderen als Summe nicht 1 ergeben. Dies liegt daran, dass die einzelnen Wahrscheinlichkeiten noch nicht normalisiert wurden, weshalb dies folgend noch gemacht wird:

In [7]:
probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

Das Array `probabilities` sieht nun wie folgt aus:

In [8]:
print(probabilities)

[0.32706751 0.14696091 0.39948116 0.12649042]


Jetzt ist ersichtlich, dass diese Werte den Werten aus <a href="#tab-1">Tabelle 1</a> (Spalte "$\varepsilon = 0.1$") entsprechen und als Summe auch 1 (bzw. 0.999999...) ergeben.

Die Implementation ist nun so weit umgesetzt, dass die Wahrscheinlichkeiten basierend auf den Scores, dem Privacy-Budget und der Sensitivität berechnet und im Array `probabilities` gespeichert werden. Als letzter Schritt muss nun noch der eigentliche Mechanismus für die Abfrage der häufigsten Krankheit umgesetzt werden. Der Mechanismus soll als Ergebnis eine der Optionen unter Berücksichtigung der vorab berechneten Wahrscheinlichkeiten zurückgeben. Dafür wird die Funktion `random.choice` der Numpy-Bibliothek verwendet (Infos zu dieser Funktion: <a href="https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html">hier</a>).

In [9]:
def dp_mostCommonDisease(diseases, probabilities):
    return np.random.choice(diseases, 1, p=probabilities)

Übersetzt heisst der Funktionsaufruf `np.random.choice(diseases, 1, p=probabilities)`:  
"Wähle aus allen möglichen 'diseases' zufällig '1' aus, basierend auf den Wahrscheinlichkeiten 'p=probabilities'.

Der Parameter "diseases" entspricht einem Array von allen möglichen Optionen, in derselben Reihenfolge, wie auch die Wahrscheinlichkeiten im Array `probabilities` angeordnet sind. Dieses Array wird nachfolgend definiert:

In [10]:
diseases = ["Diabetes",
            "Hepatitis",
            "Grippe",
            "HIV"]

Der Mechanismus `dp_mostCommonDisease` erfüllt die Eigenschaften der Differential Privacy, entsprechend ist diese nicht mehr deterministisch, sondern probabilistisch. Nachfolgend wird der Mechanismus 10 mal ausgeführt:

In [11]:
for i in range(10):
    print(dp_mostCommonDisease(diseases, probabilities))

['HIV']
['Diabetes']
['Hepatitis']
['Diabetes']
['Hepatitis']
['Grippe']
['Grippe']
['Diabetes']
['Grippe']
['Diabetes']


Es ist ersichtlich, dass die Ausgabe beim mehrmaligen Ausführen nicht immer gleich ist und die Ergebnisse nach den berechneten Wahrscheinlichkeiten verteilt sind. Weiter ist es wichtig zu erwähnen, dass auch hier die Komposition des Privacy-Budgets beim mehrmaligen Ausführen des Mechanismus berücksichtigt werden muss (siehe "Wichtige Erkenntnis" im [Notebook 3: Der Laplace Mechanismus](./3_Laplace-Mechanismus.ipynb)).

<div class="alert alert-danger">
    <h2>Übung: Exponentieller Mechanismus</h2>
    <br />
Ändere das Epsilon in der obigen Implementation auf unterschiedliche Werte ab und beobachte, wie sich die Wahrscheinlichkeiten und Ausgaben verändern. Wähle auch ein Epsilon von 0.0 sowie 1.0 und vergleiche die Wahrscheinlichkeiten mit den Werten in <a href="#tab-1">Tabelle 1</a>.
<br />
<br />
<i>Hinweis: Eine Musterlösung ist im <a href="./7_Musterlösungen-der-Übungen.ipynb">Notebook 7: Musterlösungen der Übungen</a> zu finden.</i>
</div>

## Realitätsnahes Beispiel vom exponentiellen Mechanismus

Zur Veranschaulichung wird ein weiteres Beispiel zum exponentiellen Mechanismus gegeben. Es werden die Daten zu den Berufsgruppen von 37'080 Personen aus Neuseeland ausgewertet. 

| Industry Code | Industry Name |
| :- | :- |
| CC32 | Pulp, Paper and Converted Paper Product Manufacturing |
| CC321 | Pulp, Paper and Converted Paper Product Manufacturing |
| GH134 | Pharmaceutical and Other Store Based Retailing |
| GH11 | Motor Vehicle and Motor Vehicle Parts and Fuel Retailing |
| EE13 | Construction Services |
| ... | ... |
<br>
<a name="tab-2"><center>Tabelle 2: Daten zu den Berufsgruppen von Personen</center></a>

Es soll nun die Berufsgruppe ausgegeben werden, in welcher die meisten Personen arbeiten. Nachfolgend werden die Daten geladen und in `data` gespeichert. Weiter wird im Array `onlyProfessionGroups` nur die zweite Spalte der Tabelle gespeichert, da die erste Spalte nicht benötigt wird. Zudem wird in `listOfProfessionGroups` eine Liste aller Berufsgruppen gespeichert.

In [12]:
import csv
import collections

with open('./src/profession-groups-nz.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile, delimiter=';'))

#Nur die zweite Spalte mit den Berufsgruppen in einem Array zwischenspeichern
onlyProfessionGroups = []
for x in data:
    onlyProfessionGroups.append(x[1])

#Liste aller Berufsgruppen
listOfProfessionGroups = list(dict.fromkeys(onlyProfessionGroups))

Als Bewertungsfunktion wird in diesem Beispiel wiederum die Anzahl der einzelnen Berufsgruppen gewählt. Es wird die Zählfunktion implementiert, welche die Anzahl Personen pro Berufsgruppe zurückgibt.

In [13]:
def countProfessionGroup(data):   
    #Die Anzahl Einträge pro Berufsgruppe als Collection zurückgeben
    return collections.Counter(onlyProfessionGroups)

Nun werden die Wahrscheinlichkeiten pro Berufsgruppe berechnet und normalisiert in `probabilities` gespeichert.

In [14]:
epsilon = 0.1
sensitivity = 1

probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in countProfessionGroup(data).values()]

#Normalisieren der Wahrscheinlichkeiten
probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

Der Mechanismus `dp_mostCommonProfessionGroup` gibt nun basierend auf den berechneten Wahrscheinlichkeiten die häufigste Berufsgruppe an.

In [15]:
def dp_mostCommonProfessionGroup(professionGroups, probabilities):
    return np.random.choice(professionGroups, 1, p=probabilities)

print("Resultat mit DP: " + str(dp_mostCommonProfessionGroup(listOfProfessionGroups, probabilities)))

Resultat mit DP: ['Public Order, Safety and Regulatory Services']


Zum Vergleich nachfolgend noch das tatsächliche Resultat (ohne Differential Privacy).

In [16]:
print ("Resultat ohne DP:",countProfessionGroup(data).most_common(1))

Resultat ohne DP: [('Public Order, Safety and Regulatory Services', 682)]


Es ist ersichtlich, dass offenbar der Mechanismus dasselbe Resultat liefert, als die Funktion ohne Differential Privacy. Die wichtige Frage ist nun aber, wie zuverlässig der Mechanismus das richtige Resultat liefert.

Wir führen den Mechanismus 1000 mal aus und vergleichen die Resultate.

In [17]:
samples = []

for i in range(1000):
    samples.append(str(dp_mostCommonProfessionGroup(listOfProfessionGroups, probabilities)))

for key, value in collections.Counter(samples).items():
    print(str(key) + ": " + str(value))

['Public Order, Safety and Regulatory Services']: 949
['Rental and Hiring Services (except Real Estate)']: 1
['Polymer Product and Rubber Product Manufacturing']: 8
['Primary Metal and Metal Product Manufacturing']: 4
['Printing']: 3
['Non-Metallic Mineral Product Manufacturing']: 7
['Forestry and Logging']: 2
['Transport Equipment Manufacturing']: 3
['Petroleum and Coal Product Manufacturing']: 4
['Fabricated Metal Product Manufacturing']: 7
['Pulp, Paper and Converted Paper Product Manufacturing']: 3
['Beverage and Tobacco Product Manufacturing']: 6
['Road Transport']: 2
['Wood Product Manufacturing']: 1


Es ist ersichtlich, dass die korrekte Aussage in ~950 / 1000 Fällen gemacht wurde. Der Mechanismus funktioniert also sehr zuverlässig.

## Kontrollfragen

* Es soll aus einer Datensammlung der häufigste Zivilstand ausgegeben werden. Welcher Mechanismus ist dafür am besten geeignet?
* Was könnte für den oben genannten Fall als Bewertungsfunktion verwendet werden?

_Hinweis: Eine Musterlösung der Kontrollfragen ist im nächsten Notebook zu finden._

***Hier geht es zum nächsten Notebook dieser Serie: [Notebook 6: Anwendungsfälle und Grenzen der Differential Privacy](./6_Anwendungsfälle-Grenzen.ipynb)***